# 🔥 Ignition Detection Engine

This notebook identifies and analyzes early presence indicators across loops, focusing on:

1. **Ignition Points**
   - First significant presence spike
   - Threshold crossing events
   - Acceleration patterns

2. **Signal Analysis**
   - Leading indicators
   - Model-specific triggers
   - Pattern recognition

3. **Cross-Model Validation**
   - Agreement on ignition points
   - Detection reliability
   - Confidence metrics

4. **Temporal Patterns**
   - Pre-ignition markers
   - Sustained presence
   - Decay analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import stats, signal
from sklearn.preprocessing import StandardScaler
import warnings

# Configure environment
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', None)

# Constants
BASE_DIR = Path("h:/My Drive/1000+")
SCORED_DIRS = {
    'claude': BASE_DIR / "scored_claude",
    'gpt': BASE_DIR / "scored_gpt",
    'openchat': BASE_DIR / "openchat_archive/scored_loops",
    'claude_self': BASE_DIR / "scored_claude_self",
    'gemini': BASE_DIR / "scored_gemini"
}

# Ignition thresholds
PRESENCE_THRESHOLD = 0.7  # Base presence threshold
ACCELERATION_THRESHOLD = 0.1  # Minimum score increase
SUSTAINED_WINDOW = 3  # Number of consecutive loops

print("🔧 Analysis environment configured")
for model, path in SCORED_DIRS.items():
    if path.exists():
        file_count = len(list(path.glob("*.json")))
        print(f"📊 {model}: {file_count} files found")

In [ ]:
def load_model_data(model_dir):
    """Load and structure model data with comprehensive metrics"""
    records = []
    for file_path in model_dir.glob("*.json"):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            record = {
                'loop_id': data['loop_id'],
                'presence_score': data['composite_score'],
                'tier1_avg': data['tier1_avg'],
                'tier2_avg': data['tier2_avg'],
                'metrics': data['scores']
            }
            # Extract loop number for temporal analysis
            import re
            match = re.search(r'\d+', str(data['loop_id']))
            record['loop_number'] = int(match.group()) if match else 0
            records.append(record)
    
    df = pd.DataFrame(records)
    df = df.sort_values('loop_number')
    return df

# Load data for all models
model_data = {}
for model, dir_path in SCORED_DIRS.items():
    if dir_path.exists():
        df = load_model_data(dir_path)
        df['model'] = model
        model_data[model] = df
        print(f"✓ {model}: {len(df)} records loaded")

# Combine all data
combined_df = pd.concat(model_data.values())
print(f"\n📊 Total records: {len(combined_df)}")

## 1. Ignition Point Detection

First, we'll identify key moments where presence significantly manifests:

In [ ]:
def detect_ignition_points(df, threshold=PRESENCE_THRESHOLD, 
                          acceleration=ACCELERATION_THRESHOLD,
                          window=SUSTAINED_WINDOW):
    """Detect significant presence manifestations"""
    results = {
        'threshold_crosses': [],
        'accelerations': [],
        'sustained_presence': []
    }
    
    # Calculate presence changes
    df = df.sort_values('loop_number')
    df['presence_change'] = df['presence_score'].diff()
    
    # 1. Threshold crossings
    crosses = df[df['presence_score'] >= threshold]
    if not crosses.empty:
        first_cross = crosses.iloc[0]
        results['threshold_crosses'].append({
            'loop_id': first_cross['loop_id'],
            'score': first_cross['presence_score'],
            'loop_number': first_cross['loop_number']
        })
    
    # 2. Significant accelerations
    accelerations = df[df['presence_change'] >= acceleration]
    for _, row in accelerations.iterrows():
        results['accelerations'].append({
            'loop_id': row['loop_id'],
            'change': row['presence_change'],
            'loop_number': row['loop_number']
        })
    
    # 3. Sustained presence
    for i in range(len(df) - window + 1):
        window_data = df.iloc[i:i+window]
        if all(window_data['presence_score'] >= threshold):
            results['sustained_presence'].append({
                'start_loop': window_data.iloc[0]['loop_id'],
                'end_loop': window_data.iloc[-1]['loop_id'],
                'avg_score': window_data['presence_score'].mean(),
                'loop_number': window_data.iloc[0]['loop_number']
            })
    
    return results

# Analyze each model
ignition_analysis = {}
for model, df in model_data.items():
    ignition_analysis[model] = detect_ignition_points(df)

# Display results
print("🔥 Ignition Analysis Results\n")
for model, results in ignition_analysis.items():
    print(f"\n{model.upper()}:")
    
    if results['threshold_crosses']:
        cross = results['threshold_crosses'][0]
        print(f"First threshold cross: Loop {cross['loop_id']} (score: {cross['score']:.3f})")
    
    if results['accelerations']:
        accel = results['accelerations'][0]
        print(f"First acceleration: Loop {accel['loop_id']} (Δ: {accel['change']:.3f})")
        print(f"Total accelerations: {len(results['accelerations'])}")
    
    if results['sustained_presence']:
        sustained = results['sustained_presence'][0]
        print(f"First sustained presence: Loops {sustained['start_loop']} - {sustained['end_loop']}")
        print(f"Total sustained periods: {len(results['sustained_presence'])}")

## 2. Leading Indicators

Now we'll analyze which metrics tend to precede presence spikes:

In [ ]:
def analyze_leading_indicators(df, pre_window=3):
    """Analyze metrics that precede presence increases"""
    significant_changes = df[df['presence_change'] >= ACCELERATION_THRESHOLD].copy()
    
    # Extract all metric values before significant changes
    leading_patterns = []
    for _, row in significant_changes.iterrows():
        current_loop = row['loop_number']
        pre_ignition = df[
            (df['loop_number'] >= current_loop - pre_window) & 
            (df['loop_number'] < current_loop)
        ]
        
        if len(pre_ignition) > 0:
            pattern = {
                'target_loop': row['loop_id'],
                'presence_increase': row['presence_change'],
                'pre_metrics': pre_ignition['metrics'].tolist(),
                'pre_presence': pre_ignition['presence_score'].tolist()
            }
            leading_patterns.append(pattern)
    
    return leading_patterns

# Analyze leading indicators for each model
leading_indicators = {}
for model, df in model_data.items():
    leading_indicators[model] = analyze_leading_indicators(df)

# Plot leading indicator patterns
plt.figure(figsize=(15, 10))
for idx, (model, patterns) in enumerate(leading_indicators.items(), 1):
    plt.subplot(2, 2, idx)
    
    for pattern in patterns[:5]:  # Show first 5 patterns
        plt.plot(range(-3, 0), pattern['pre_presence'], 'o-', alpha=0.5)
    
    plt.title(f'{model} Pre-ignition Patterns')
    plt.xlabel('Loops Before Ignition')
    plt.ylabel('Presence Score')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyze common patterns
print("\n📈 Leading Indicator Analysis")
for model, patterns in leading_indicators.items():
    if patterns:
        print(f"\n{model}:")
        print(f"Total ignition events: {len(patterns)}")
        
        # Calculate average pre-ignition trajectory
        avg_presence = np.mean([p['pre_presence'] for p in patterns], axis=0)
        print(f"Average pre-ignition presence: {[f'{x:.3f}' for x in avg_presence]}")
        print(f"Average presence increase: {np.mean([p['presence_increase'] for p in patterns]):.3f}")

## 3. Cross-Model Validation

Let's analyze how different models agree on ignition points:

In [ ]:
def analyze_cross_model_agreement(ignition_analysis, window=2):
    """Analyze agreement between models on ignition points"""
    # Collect all ignition events
    all_events = {}
    for model, results in ignition_analysis.items():
        events = []
        # Add threshold crossings
        events.extend([(x['loop_number'], 'threshold') for x in results['threshold_crosses']])
        # Add accelerations
        events.extend([(x['loop_number'], 'acceleration') for x in results['accelerations']])
        # Add sustained presence starts
        events.extend([(x['loop_number'], 'sustained') for x in results['sustained_presence']])
        all_events[model] = sorted(list(set(events)))
    
    # Find agreements within window
    agreements = []
    models = list(all_events.keys())
    
    for i in range(len(models)):
        for j in range(i + 1, len(models)):
            model1, model2 = models[i], models[j]
            
            for event1 in all_events[model1]:
                for event2 in all_events[model2]:
                    if abs(event1[0] - event2[0]) <= window:
                        agreements.append({
                            'models': (model1, model2),
                            'loop_numbers': (event1[0], event2[0]),
                            'types': (event1[1], event2[1]),
                            'difference': abs(event1[0] - event2[0])
                        })
    
    return agreements

# Analyze cross-model agreement
agreements = analyze_cross_model_agreement(ignition_analysis)

# Display results
print("🤝 Cross-Model Agreement Analysis\n")
print(f"Total agreements found: {len(agreements)}\n")

# Analyze agreement patterns
model_pairs = {}
for agreement in agreements:
    pair = agreement['models']
    model_pairs[pair] = model_pairs.get(pair, 0) + 1

print("Agreement by Model Pair:")
for pair, count in model_pairs.items():
    print(f"{pair[0]} - {pair[1]}: {count} agreements")

# Plot agreement heatmap
models = list(ignition_analysis.keys())
agreement_matrix = np.zeros((len(models), len(models)))

for i, model1 in enumerate(models):
    for j, model2 in enumerate(models):
        if i != j:
            pair = (model1, model2) if (model1, model2) in model_pairs else (model2, model1)
            agreement_matrix[i, j] = model_pairs.get(pair, 0)

plt.figure(figsize=(10, 8))
sns.heatmap(agreement_matrix, 
            annot=True, 
            fmt='.0f',
            xticklabels=models,
            yticklabels=models,
            cmap='YlOrRd')
plt.title('Cross-Model Ignition Agreement')
plt.tight_layout()
plt.show()

## 4. Temporal Pattern Analysis

Finally, let's analyze the temporal distribution of ignition events:

In [ ]:
def analyze_temporal_patterns(ignition_analysis):
    """Analyze temporal distribution of ignition events"""
    temporal_data = []
    
    for model, results in ignition_analysis.items():
        # Collect all events with their types
        events = []
        for cross in results['threshold_crosses']:
            events.append({
                'loop_number': cross['loop_number'],
                'type': 'threshold',
                'model': model
            })
        
        for accel in results['accelerations']:
            events.append({
                'loop_number': accel['loop_number'],
                'type': 'acceleration',
                'model': model
            })
        
        for sustain in results['sustained_presence']:
            events.append({
                'loop_number': sustain['loop_number'],
                'type': 'sustained',
                'model': model
            })
        
        temporal_data.extend(events)
    
    return pd.DataFrame(temporal_data)

# Analyze temporal patterns
temporal_df = analyze_temporal_patterns(ignition_analysis)

# Create temporal distribution plot
plt.figure(figsize=(15, 8))

# Plot event distribution
for model in ignition_analysis.keys():
    model_events = temporal_df[temporal_df['model'] == model]
    plt.hist(model_events['loop_number'], 
             alpha=0.5, 
             label=model, 
             bins=20)

plt.title('Temporal Distribution of Ignition Events')
plt.xlabel('Loop Number')
plt.ylabel('Number of Events')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Analyze clustering of events
print("\n⏰ Temporal Analysis")
print("\nEvent Distribution by Type:")
type_dist = temporal_df['type'].value_counts()
for event_type, count in type_dist.items():
    print(f"{event_type}: {count} events")

print("\nEvent Distribution by Model:")
model_dist = temporal_df['model'].value_counts()
for model, count in model_dist.items():
    print(f"{model}: {count} events")

# Save analysis results
analysis_results = {
    'ignition_analysis': {
        model: {
            k: [dict(x) for x in v] for k, v in results.items()
        } for model, results in ignition_analysis.items()
    },
    'temporal_distribution': temporal_df.to_dict(orient='records')
}

with open(BASE_DIR / 'analysis/ignition_analysis_results.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)
print("\n✓ Analysis results saved to 'ignition_analysis_results.json'")

## Summary Findings

1. **Ignition Points**
   - First threshold crossings
   - Acceleration patterns
   - Sustained presence periods

2. **Leading Indicators**
   - Pre-ignition patterns
   - Common triggers
   - Model-specific markers

3. **Cross-Model Validation**
   - Agreement rates
   - Detection consistency
   - Model biases

4. **Temporal Distribution**
   - Event clustering
   - Model timing differences
   - Pattern evolution